In [1]:
using Revise, Oscar # execute to start up Oscar

┌ Info: Precompiling Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13]
└ @ Base loading.jl:1423
[ Info: Regenerating /home/zach/.julia/packages/Singular/N1JNo/src/libraryfuncdictionary.jl


 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


### Exercise 1: 

Consider the ideal $I$ generated by the polynomial $f = (x^2 - y^5)\cdot((x^3-1)^2-y^3)$. 

 - What is the singular locus of the affine variety defined by $I$? 
 - What are the components of the singular locus? How many are there?

### Discussion

We define the needed ring and the polynomial $f$ in it. Then we apply the Jacobian criterion 
to determine the singular locus and compute a primary decomposition of the result.

In [8]:
R,(x,y)=QQ["x","y"]
f = (x^2 - y^5)*((x^3-1)^2-y^3)
Jf = ideal(R, minors(jacobi_matrix(f),1))
Jff = Jf + ideal(R,f)
primary_decomposition(radical(Jff))

6-element Vector{Tuple{MPolyIdeal{fmpq_mpoly}, MPolyIdeal{fmpq_mpoly}}}:
 (ideal(y, x), ideal(x, y))
 (ideal(y^6 + y^3 + 1, x - y^4 - y), ideal(y^6 + y^3 + 1, x - y^4 - y))
 (ideal(y^9 - y^6 + 2*y^3 - 1, x - y^7 + y^4 - y), ideal(y^9 - y^6 + 2*y^3 - 1, x - y^7 + y^4 - y))
 (ideal(y^15 - 2*y^9 + y^3 - 1, x - y^10 + y^4), ideal(y^15 - 2*y^9 + y^3 - 1, x - y^10 + y^4))
 (ideal(y, x - 1), ideal(y, x - 1))
 (ideal(y, x^2 + x + 1), ideal(y, x^2 + x + 1))

Some of the components are already points with rational coordinates. Others consist of several points 
but with complex coordinates. When working over $\mathbb Q$, these can not be decomposed further, 
but we can try to add algebraic elements to $\mathbb Q$ until we eventually arrive at a splitting 
of all components into geometric points. For example, as a first step, we could adjoin a nineth root 
of unity. Over $\mathbb Q[\zeta_9]$ the second component above then splits up into six points:

In [47]:
P, t = QQ["t"]
kk = splitting_field(t^9-1)
Rext, (X,Y) = kk["x", "y"]
Iext = ideal(Rext, [Y^6+Y^3+1, X-Y^4-Y])
dec = primary_decomposition(Iext)

6-element Vector{Tuple{MPolyIdeal{AbstractAlgebra.Generic.MPoly{nf_elem}}, MPolyIdeal{AbstractAlgebra.Generic.MPoly{nf_elem}}}}:
 (ideal(y - x^4, x + (-x^3 - 1)*y), ideal(y - x^4, x + (-x^3 - 1)*y))
 (ideal(y - x, x - x^4 - x), ideal(y - x, x - x^4 - x))
 (ideal(y + x^4 + x, x + (-x^3 - 1)*y), ideal(y + x^4 + x, x + (-x^3 - 1)*y))
 (ideal(y - x^2, x + x^3*y), ideal(y - x^2, x + x^3*y))
 (ideal(y - x^5, x + x^3*y), ideal(y - x^5, x + x^3*y))
 (ideal(y + x^5 + x^2, x + x^3*y), ideal(y + x^5 + x^2, x + x^3*y))

It seems a bit strange that $\zeta_9$ does not appear in the above expressions. But we can verify that 
each one is in fact a simple point by computing the dimension of the associated quotient:

In [51]:
[singular_vdim(comp[1]) for comp in dec]

6-element Vector{Int64}:
 1
 1
 1
 1
 1
 1

### Exercise 2:

For the same ideal $I$ from Exercise 1 there are (among others) singularities at $p = (0, 1)$ and $q = (1, 0)$.

 - What are the multiplicities, the Milnor numbers and the Tjurina numbers of these local singularities?
 - What are the global Milnor- and Tjurina numbers of the affine variety defined by $I$?

### Discussion

Let us first verify that the two points are indeed singular points of the hypersurface $X$ defined by $f$. 
For $q$ we can read this off the primary decomposition over $\mathbb Q$ above: It appears as the second last component. Let us check whether for $p$ all generators of the ideal `Jff` really vanish:

In [52]:
[g(0, 1) for g in gens(Jff)]

3-element Vector{fmpq}:
 0
 3
 0

No, they don't! So $p$ is not a singular point. Sorry, this was a mistake on our side when posing the exercise! 
Instead, let us consider $p = (0,0)$, the origin and start out by computing the local Milnor number at that point.

In [53]:
# For comparison, we first compute the global Milnor number (in R)
J = ideal(R, minors(jacobi_matrix(f), 1))
singular_vdim(J)

73

In [54]:
# compute the local Milnor number at the origin (0,0) with standard basis 
# and local orderings
o=negdegrevlex(gens(R))
S=std_basis(J,o)    


2-element Vector{fmpq_mpoly}:
 x
 y^4

It is now easy to see from the monomial diagram that the local Milnor number at the origin is $4$. 
There are several ways to automatize this counting. For instance, we could do the following:

In [58]:
### routine to collect all monomials which are not in the leading ideal of I.
function local_kbase(I::MPolyIdeal)
    R = base_ring(I)
    o = negdegrevlex(gens(R)) # Choose a local monomial ordering
    lead_I = ideal(R, std_basis(I, o)) # Compute the leading ideal of I
    d = 0
    result = elem_type(R)[] # an empty list, but with a prescribed type (advanced stuff, don't worry!)
    new_mons = [m for m in Oscar.all_monomials(R, d) if !(m in lead_I)] # a fancy way to collect monomials of degree d
    while length(new_mons) > 0 
        result = vcat(result, new_mons)
        d = d+1
        new_mons = [m for m in Oscar.all_monomials(R, d) if !(m in lead_I)] 
    end
    return result
end

# Let's apply it to J above
local_kbase(J)

4-element Vector{fmpq_mpoly}:
 1
 y
 y^2
 y^3

Now if we are only interested in the Milnor number, do this:

In [60]:
function local_milnor_number(f::MPolyElem)
    return length(local_kbase(ideal(parent(f), minors(jacobi_matrix(f), 1))))
end

# apply it to f from above:
local_milnor_number(f)

4

If we want to compute the local Milnor number of another singular point $q$ *outside* the origin, 
then, if we are to use local orderings for this, we must first apply a translation 
$T$ to the ambient space which shifts the point $q$ to the origin.

In [61]:
# This is a manual shift for q = (1, 0) by substitution. 
fshift = ((x+1)^2 - y^5)*(((x+1)^3-1)^2-y^3)
Jfsh = ideal(R, minors(jacobi_matrix(fshift),1))
singular_vdim_local(Jfsh)

2

In [63]:
# In Oscar, we can also construct the translation as a map:
T = hom(R, R, [x+1, y])
T(f) == fshift

true

All of the above has been implemented to work in the background for localized rings. 
In fact, we could have taken the following shortcut. 

In [67]:
U = complement_of_ideal(R, [1, 0])
L, _ = Localization(R, U)
Jf_loc = L(Jf)
vdim(Jf_loc) # WARNING: This last command is not in the official OSCAR!

2

But, of course, you do not see anything here anymore. 

### Exercise 3:

Let $R$ be the polynomial ring (over $\mathbf Q$ or $\mathbf C$) with $11$ variables $w_1, \dots, w_{11}$.
Consider the matrix 

$M = \begin{pmatrix} w_1 & w_2 & w_3 & w_4 \\ w_5 & w_6 & w_7 & w_8 \\ w_9 & w_{10} & w_{11} & f \end{pmatrix}$

with $f = w_1\cdot (w_1^2 - w_6^2)$.


 - What is the singular locus $\Sigma(X)$ of the variety $X$ defined by the maximal minors of $M$?
 - What is the singular locus $\Sigma(\Sigma(X))$ of that singular locus? 
 - Compare the above singular loci to the varieties defined by non-maximal minors of size $2$ or $1$. What do you see?

### Exercise 4:

A mystery ideal. Consider the ideal 

$I = \langle x^7*y + 3*x^5*y^3 - 3*x^5*y + 3*x^3*y^5 - 6*x^3*y^3 + 3*x^3*y + x*y^7 - 3*x*y^5 + 3*x*y^3 - x*y*z^2 - x*y
 x^9 + 3*x^7*y^2 - x^7*z^3 + 3*x^7*z^2 - 3*x^7*z - 2*x^7 + 3*x^5*y^4 - 3*x^5*y^2*z^3 + 9*x^5*y^2*z^2 - 9*x^5*y^2*z - 3*x^5*y^2 + 3*x^5*z^3 - 9*x^5*z^2 + 9*x^5*z + x^3*y^6 - 3*x^3*y^4*z^3 + 9*x^3*y^4*z^2 - 9*x^3*y^4*z + 6*x^3*y^2*z^3 - 18*x^3*y^2*z^2 + 18*x^3*y^2*z - 3*x^3*y^2 - 3*x^3*z^3 + 8*x^3*z^2 - 9*x^3*z + 2*x^3 - x*y^6*z^3 + 3*x*y^6*z^2 - 3*x*y^6*z + x*y^6 + 3*x*y^4*z^3 - 9*x*y^4*z^2 + 9*x*y^4*z - 3*x*y^4 - 3*x*y^2*z^3 + 9*x*y^2*z^2 - 9*x*y^2*z + 3*x*y^2 + x*z^5 - 3*x*z^4 + 4*x*z^3 - 4*x*z^2 + 3*x*z - x
 x^6*y^2 + 3*x^4*y^4 - 3*x^4*y^2 + 3*x^2*y^6 - 6*x^2*y^4 + 3*x^2*y^2 + y^8 - 3*y^6 + 3*y^4 - y^2*z^2 - y^2
 x^8*y + 3*x^6*y^3 - x^6*y*z^3 + 3*x^6*y*z^2 - 3*x^6*y*z - 2*x^6*y + 3*x^4*y^5 - 3*x^4*y^3*z^3 + 9*x^4*y^3*z^2 - 9*x^4*y^3*z - 3*x^4*y^3 + 3*x^4*y*z^3 - 9*x^4*y*z^2 + 9*x^4*y*z + x^2*y^7 - 3*x^2*y^5*z^3 + 9*x^2*y^5*z^2 - 9*x^2*y^5*z + 6*x^2*y^3*z^3 - 18*x^2*y^3*z^2 + 18*x^2*y^3*z - 3*x^2*y^3 - 3*x^2*y*z^3 + 8*x^2*y*z^2 - 9*x^2*y*z + 2*x^2*y - y^7*z^3 + 3*y^7*z^2 - 3*y^7*z + y^7 + 3*y^5*z^3 - 9*y^5*z^2 + 9*y^5*z - 3*y^5 - 3*y^3*z^3 + 9*y^3*z^2 - 9*y^3*z + 3*y^3 + y*z^5 - 3*y*z^4 + 4*y*z^3 - 4*y*z^2 + 3*y*z - y\rangle$ 
 
in the ring $R = \mathbb Q[x,y,z]$. 
What can you say about the geometry of the variety $X = V(I)$?

 * What is the dimension of $X$? Is it equidimensional? 
 * What is the singular locus of $X$? 
 * Which singularities are singularities of components of $X$ and which singularities are due to intersections of components?
 * for the isolated singularities of $X$: Can you compute the multiplicities and the Milnor numbers?
 * Can you do the same for the isolated singularities of the components?